In [1]:
import pandas as pd
import numpy as np
import json
import os
from datetime import datetime

## Clean Gasprices

In [2]:
DIR_GAS_PRICES_RAW = "data/gas_prices"
DIR_GAS_PRICES_CLEAN = "data/clean_gas_prices"
os.makedirs(DIR_GAS_PRICES_CLEAN, exist_ok=True)

# iterate over data and parse gas prices
data_frames = []
for file in os.listdir(DIR_GAS_PRICES_RAW):
    with open(f"{DIR_GAS_PRICES_RAW}/{file}", "r") as fp:
        gas_prices = json.loads(fp.read())

    timestamp = pd.to_datetime(gas_prices['chart']['result'][0]['timestamp'], unit="s")
    data = gas_prices['chart']['result'][0]['indicators']['quote'][0]
    mean_price = np.mean(np.vstack((np.array(data['low']), np.array(data['high']))), axis=0)

    data_frames.append(pd.DataFrame({
        "timestamp": timestamp,
        "gas_price": mean_price
    }))
data = pd.concat(data_frames)

# remove duplicates and filter to daterange 2017 - 2022
data = data.drop_duplicates()
data = data[(data.timestamp >= datetime(2017, 1, 1)) & (data.timestamp <= datetime(2022,1,1))]
assert data.timestamp.unique().shape[0] == data.shape[0]
data.timestamp = data.timestamp.dt.date
data.reset_index(drop=True, inplace=True)

# add dates when markets were closed and interpolate
new_timestamps = pd.DataFrame({"date": pd.date_range(start=data.timestamp.iloc[0], end=data.timestamp.iloc[-1], freq="1d")})
new_timestamps.date = new_timestamps.date.dt.date
data = pd.merge(left=new_timestamps, right=data, left_on="date", right_on="timestamp", how='left')
data.drop(columns=["timestamp"], inplace=True)
data['gas_price'] = data['gas_price'].interpolate(method="linear")

data.to_csv(f"{DIR_GAS_PRICES_CLEAN}/gas_prices.csv", index=False)